In [1]:
import numpy as np
import pandas as pd 
import requests
import faiss
import math

In [45]:
amenities = pd.read_json('amenities.json')
amenities['idAmenity'] = amenities['idAmenity'].astype(str)
amenities.columns


Index(['idAmenity', 'description', 'nameAmenity', 'categories', 'price'], dtype='object')

In [46]:
def textual_representation(row):
    return f"""idAmenity: {row['idAmenity']} nameAmenity: {row['nameAmenity']} description: {row['description']} categories: {row['categories']} price: {row['price']}   
"""

In [47]:
amenities['textual_representation'] = amenities.apply(textual_representation, axis=1)
amenities['textual_representation']

0     idAmenity: 1 nameAmenity: Alarm Clock descript...
1     idAmenity: 2 nameAmenity: Cleaning Studio desc...
2     idAmenity: 3 nameAmenity: Cleaning Suite descr...
3     idAmenity: 4 nameAmenity: Scooter Rental descr...
4     idAmenity: 5 nameAmenity: WiFi description: Hi...
5     idAmenity: 6 nameAmenity: View description: En...
6     idAmenity: 7 nameAmenity: Smoking description:...
7     idAmenity: 10 nameAmenity: Free Coffee descrip...
8     idAmenity: 12 nameAmenity: TV 55 description: ...
9     idAmenity: 13 nameAmenity: Baby Cot descriptio...
10    idAmenity: 14 nameAmenity: Breakfast descripti...
11    idAmenity: 15 nameAmenity: Early Check-in desc...
12    idAmenity: 16 nameAmenity: Extra Bed descripti...
13    idAmenity: 17 nameAmenity: In-room Spa Treatme...
14    idAmenity: 18 nameAmenity: Late Check-out desc...
15    idAmenity: 19 nameAmenity: Luxury Car Rental d...
16    idAmenity: 20 nameAmenity: Private Airport Tra...
17    idAmenity: 21 nameAmenity: Private Chef Se

In [51]:
dim = 3072
index = faiss.IndexFlatL2(dim)
embeddings = np.zeros((amenities.shape[0], dim), dtype=np.float32)
embeddings.shape

(22, 3072)

In [52]:
for i , represenrtation in enumerate(amenities['textual_representation']):
    url = 'http://localhost:11434/api/embeddings'
    body = {
        'model' : 'llama3.2',
        'prompt': represenrtation
    }
    request = requests.post(url,json=body)
    embedding = request.json()['embedding']
    embeddings[i] = np.array(embedding)

index.add(embeddings)

In [53]:
faiss.write_index(index, 'amenities.index')

In [64]:
faiss.read_index('amenities.index')

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001E3217FF570> >

In [71]:
query = ' '

request = requests.post(url,json=body)
emb = request.json()['embedding']
emb = np.array(emb).reshape(1, -1)
print(emb.shape)


(1, 3072)


In [72]:
distance, indices = index.search(emb, 10)
indices = indices.flatten().tolist()
indices

[21, 7, 6, 8, 20, 10, 14, 9, 19, 17]

In [73]:
best_matches = amenities['nameAmenity'].iloc[indices]
best_matches

21               Room Service
7                 Free Coffee
6                     Smoking
8                       TV 55
20           Breakfast Buffet
10                  Breakfast
14             Late Check-out
9                    Baby Cot
19    Airport Shuttle Service
17       Private Chef Service
Name: nameAmenity, dtype: object